## FastAPI

## Installation

In [2]:
#pip install fastapi
#pip install "uvicorn[standard]"

## Quick example

In [ ]:
from typing import Union

from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/items/{item_id}")
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}

## Asynchronous version
* In you are using third party libraries that tell you to call them with `await`, use `async def` instead of `def`.

In [ ]:
from typing import Union

from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def read_root():
    results = await some_library()
    return results

#### When to use asyncronous code?
* "Synchronous" or "sequential" code means that the computer / program follows all the steps in sequence before switching to a different task, even if those steps involve waiting.
* Asynchronous code just means that the language has a way to tell the computer / program that at some point in the code, it will have to wait for something else to finish somewhere else. Let's say that something else is called "slow-file". So, during that time, the computer can go and do some other work, while "slow-file" finishes.
* That "wait for something else" normally refers to I/O operations that are relatively "slow" (compared to the speed of the processor and the RAM memory), like waiting for:
    * the data from the client to be sent through the network
    * the data sent by your program to be received by the client through the network
    * the contents of a file in the disk to be read by the system and given to your program
    * the contents your program gave to the system to be written to disk
    * a remote API operation
    * a database operation to finish
    * a database query to return the results
    * etc.

## Run the code

In [3]:
##uvicorn main:app --reload

## Check it in your browser
* http://127.0.0.1:8000
* http://127.0.0.1:8000/items/5?q=somequery

## API docs
* http://127.0.0.1:8000/docs
* http://127.0.0.1:8000/redoc

## BaseModel, Filter, Parameters and Queries

In [ ]:
from typing import Union
from fastapi import FastAPI
from models.item import Item

app = FastAPI()

items = [
    {
        "id": 1,
        "name": "iphone",
        "price": 950,
        "is_offer": False
    },
    {
        "id": 2,
        "name": "samsung",
        "price": 750,
        "is_offer": True
    }
]

@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.post("/items")
def create_item(item: Item):
    items.append(item)
    return items

@app.get("/items")
def get_items():
    return items

@app.get("/items/")
def get_items_by_name(name: str):
    return list(filter(lambda item: item['name'] == name, items))

@app.get("/items/{item_id}")
def read_item(item_id: int, q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}

@app.put("/items/{item_id}")
def update_item(item_id: int, item: Item):
    return {
        "id": item_id,
        "name": item.name,
        "price": item.price,
        "is_offer": item.is_offer
        }

@app.put("/items/update/{item_id}")
def second_update_item(item_id: int, item: Item):
    for index, item in enumerate(items):
        if item["id"] == item_id:
            item[index]["name"] = item.name
            item[index]["price"] = item.price
            item[index]["is_offer"] = item.is_offer
    return items

@app.delete("/items/{item_id}")
def eliminate_item(item_id: int):
    for item in items:
        if item["id"] ==item_id:
            items.remove(item)
    return items

## Schemas
* Model definition with BaseModel from Pydantic.
* Type validation.

In [5]:
from typing import Union, Optional
from pydantic import BaseModel

class Item(BaseModel):
    id: int
    name: str
    price: Optional[float] = None
    is_offer: Union[bool, None] = None

## Routers: when we have more than one backend API
* For example, item API and user API
* We will create routers/item.py and routers/user.py
* Below is item.py

In [ ]:
from fastapi import APIRouter, Path
from models.item import Item
from typing import Union

router = APIRouter()

items = [
    {
        "id": 1,
        "name": "iphone",
        "price": 950,
        "is_offer": False
    },
    {
        "id": 2,
        "name": "samsung",
        "price": 750,
        "is_offer": True
    }
]

@router.get("/items/{item_id}")
def read_item(item_id: int = Path(ge=0), q: Union[str, None] = None):
    return {"item_id": item_id, "q": q}

@router.put("/items/{item_id}")
def update_item(item_id: int, item: Item):
    return {
        "id": item_id,
        "name": item.name,
        "price": item.price,
        "is_offer": item.is_offer
        }

@router.put("/items/update/{item_id}")
def second_update_item(item_id: int, item: Item):
    for index, item in enumerate(items):
        if item["id"] == item_id:
            item[index]["name"] = item.name
            item[index]["price"] = item.price
            item[index]["is_offer"] = item.is_offer
    return items

@router.get("/items/")
def get_items_by_name(name: str):
    return list(filter(lambda item: item['name'] == name, items))

@router.get("/items")
def get_items():
    return items

@router.post("/items")
def create_item(item: Item):
    items.append(item)
    return items

@router.delete("/items/{item_id}")
def eliminate_item(item_id: int):
    for item in items:
        if item["id"] ==item_id:
            items.remove(item)
    return items

After this, we can remove the endpoints from main.py and include there:
* app.include_router(item_router)